In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark2 =SparkSession.builder \
  .master('local[*]') \
  .appName('conversions') \
  .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.23.2') \
  .config('spark.executor.memory', '4G') \
  .config('spark.driver.memory', '45G') \
  .config('spark.driver.maxResultSize', '10G') \
  .config("spark.memory.offHeap.size","16g") \
  .getOrCreate() 
#   .config("spark.executor.memory", "70g")
#   .config("spark.memory.offHeap.size","8g")  \
#   .config("spark.driver.memory", "8g") \
sqlContext = SQLContext(spark2)

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d29e9d3f-b4a3-4117-a703-9d6d1cb8b5ad;1.0
	confs: [default]
	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.23.2 in central
:: resolution report :: resolve 337ms :: artifacts dl 21ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.23.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retri

In [2]:
 spark2.conf.get('spark.driver.maxResultSize')

'10G'

In [3]:
!gsutil cp gs://nft_sql_lite_db/nfts.sqlite /tmp/nfts.sqlite

Copying gs://nft_sql_lite_db/nfts.sqlite...
\ [1 files][  6.9 GiB/  6.9 GiB]   49.3 MiB/s                                   
Operation completed over 1 objects/6.9 GiB.                                      


In [4]:
!ls /tmp/nfts.sqlite

/tmp/nfts.sqlite


In [5]:
import sqlite3
import csv
import pandas as pd

db_path = '/tmp/nfts.sqlite'

In [9]:
query = 'SELECT * from current_owners'

conn = sqlite3.connect(db_path)

a_pandas_df = pd.read_sql_query(query, conn)

In [10]:
spark_current_owner = sqlContext.createDataFrame(data=a_pandas_df)

In [11]:
spark_current_owner.printSchema()

root
 |-- nft_address: string (nullable = true)
 |-- token_id: string (nullable = true)
 |-- owner: string (nullable = true)



In [12]:
spark_current_owner.show(5)

22/04/12 12:40:33 WARN org.apache.spark.scheduler.TaskSetManager: Stage 0 contains a task of very large size (173673 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------+--------------------+
|         nft_address|token_id|               owner|
+--------------------+--------+--------------------+
|0x00000000000b7F8...|       0|0xb776cAb26B9e6Be...|
|0x00000000000b7F8...|       1|0x8A73024B39A4477...|
|0x00000000000b7F8...|      10|0x5e5C817E9264B46...|
|0x00000000000b7F8...|      11|0x8376f63c13b99D3...|
|0x00000000000b7F8...|      12|0xb5e34552F32BA92...|
+--------------------+--------+--------------------+
only showing top 5 rows



In [ ]:
spark_current_owner.write.format('bigquery').option("temporaryGcsBucket","temp_bucket_temp").option('table', 'ExtendedSqlLiteDataset.nft_current_owners').partitionBy("nft_address").mode('overwrite').save()

In [2]:
query = 'SELECT * from nfts'

conn = sqlite3.connect(db_path)

pandas_df = pd.read_sql_query(query, conn)

DatabaseError: Execution failed on sql 'SELECT * from nfts': no such table: nfts

In [ ]:
spark_nfts = sqlContext.createDataFrame(data=pandas_df)

In [ ]:
spark_nfts.printSchema()
spark_nfts.show(5)

In [ ]:
spark_nfts.write.format('bigquery').option("temporaryGcsBucket","temp_bucket_temp").option('table', 'ExtendedSqlLiteDataset.nfts').mode('overwrite').save()

In [2]:
# query = 'SELECT * from transfers'

conn = sqlite3.connect(db_path)

# pandas_df = pd.read_sql_query(query, conn)

In [3]:
cur = conn.execute("SELECT * from transfers")

rows = cur.fetchall()

In [7]:
table_columns = ['eventId', 'transaction_hash', 'block_number', 'nft_address', 'token_id', 'from_address', 'to_address', 'transaction_value', 'timestamp']

with open("transfers.csv", "w", newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(table_columns)
    writer.writerows(rows)

In [3]:
transfer_data = pd.read_csv('transfers.csv')

transfer_data.head()

,eventId,transaction_hash,block_number,nft_address,token_id,from_address,to_address,transaction_value,timestamp
0,cd816651-56b2-4ed9-887c-c83de732428d,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,6682934142305278177002530107138320246486863356...,0x25f1d709b329C7349b8209851E90eFa3a7f60178,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.0,1627776481
1,82cc5228-eb80-4e0d-9f6f-e644dec3ab06,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,1198507443711741845273832738859307575893198744...,0x0737E7162C88E9FBB963334e4Bfe6e97447fF811,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.0,1627776481
2,6e1f9cc4-d1df-4a6b-972d-a20765beb326,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,1541220344281436903042142177888467401468454835...,0xA0fEc186c4b3FAd39ca373dFdd52E79C2495F2c3,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.0,1627776481
3,43dc27a7-a72d-4894-809c-e868de05f7ee,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,7993840355029527745913866651030234226423689128...,0x90bE22069560A9bAd99543B4A922d1FE34Ae84E9,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.0,1627776481
4,47b7839b-9b87-442d-b2c1-9ebedcad8e06,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,1123719749142984581497870712442861055686734521...,0xA8E9C42Ebb915EeEC5080B7Bcc5D844475595091,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.0,1627776481


In [ ]:
spark_transfer = sqlContext.createDataFrame(data=transfer_data)

In [6]:
spark_transfer.printSchema()
spark_transfer.show(5)

root
 |-- eventId: string (nullable = true)
 |-- transaction_hash: string (nullable = true)
 |-- block_number: long (nullable = true)
 |-- nft_address: string (nullable = true)
 |-- token_id: string (nullable = true)
 |-- from_address: string (nullable = true)
 |-- to_address: string (nullable = true)
 |-- transaction_value: double (nullable = true)
 |-- timestamp: long (nullable = true)



22/04/12 13:59:04 WARN org.apache.spark.scheduler.TaskSetManager: Stage 0 contains a task of very large size (222171 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------+
|             eventId|    transaction_hash|block_number|         nft_address|            token_id|        from_address|          to_address|transaction_value| timestamp|
+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------+
|cd816651-56b2-4ed...|0x0c73daad5e6946e...|    12936373|0x629A673A8242c2A...|66829341423052781...|0x25f1d709b329C73...|0xC69b4c6fFDBaF84...|              0.0|1627776481|
|82cc5228-eb80-4e0...|0x0c73daad5e6946e...|    12936373|0x629A673A8242c2A...|11985074437117418...|0x0737E7162C88E9F...|0xC69b4c6fFDBaF84...|              0.0|1627776481|
|6e1f9cc4-d1df-4a6...|0x0c73daad5e6946e...|    12936373|0x629A673A8242c2A...|15412203442814369...|0xA0fEc186c4b3FAd...|0xC69b4c6fFDBaF84...|          

In [2]:
df = sqlContext.read.format("csv").option("header","true").option("delimiter", ",").load("gs://nft_csvingestion/transfers.csv")
df.show(10)

+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------+
|             eventId|    transaction_hash|block_number|         nft_address|            token_id|        from_address|          to_address|transaction_value| timestamp|
+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------+
|cd816651-56b2-4ed...|0x0c73daad5e6946e...|    12936373|0x629A673A8242c2A...|66829341423052781...|0x25f1d709b329C73...|0xC69b4c6fFDBaF84...|                0|1627776481|
|82cc5228-eb80-4e0...|0x0c73daad5e6946e...|    12936373|0x629A673A8242c2A...|11985074437117418...|0x0737E7162C88E9F...|0xC69b4c6fFDBaF84...|                0|1627776481|
|6e1f9cc4-d1df-4a6...|0x0c73daad5e6946e...|    12936373|0x629A673A8242c2A...|15412203442814369...|0xA0fEc186c4b3FAd...|0xC69b4c6fFDBaF84...|          

In [3]:
df.write.format('bigquery').option("temporaryGcsBucket","temp_bucket_temp").option('table', 'ExtendedSqlLiteDataset.transfers').partitionBy("nft_address").mode('overwrite').save()

In [11]:
spark_transfer.write.format('bigquery').option("temporaryGcsBucket","temp_bucket_temp").option('table', 'ExtendedSqlLiteDataset.transfers').partitionBy("nft_address").mode('overwrite').save()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40817)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:40817)

In [6]:
conn = sqlite3.connect(db_path)

In [7]:
cur = conn.execute("SELECT * from ownership_transitions")

rows = cur.fetchall()

In [8]:
table_columns = ['from_address', 'to_address', 'num_transitions']

with open("ownership_transitions.csv", "w", newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(table_columns)
    writer.writerows(rows)

In [9]:
df = sqlContext.read.format("csv").option("header","true").option("delimiter", ",").load("gs://nft_csvingestion/ownership_transitions.csv")
df.show(10)

+--------------------+--------------------+---------------+
|        from_address|          to_address|num_transitions|
+--------------------+--------------------+---------------+
|0x000000000000000...|0x000000000000000...|          83548|
|0x000000000000000...|0x000000000000000...|              1|
|0x000000000000000...|0x000000000000000...|           5531|
|0x000000000000000...|0x000000000000000...|              1|
|0x000000000000000...|0x000000000000000...|              1|
|0x000000000000000...|0x000000000000000...|              1|
|0x000000000000000...|0x000000000000000...|              1|
|0x000000000000000...|0x000000000000000...|              1|
|0x000000000000000...|0x000000000000000...|              4|
|0x000000000000000...|0x000000000000000...|          19758|
+--------------------+--------------------+---------------+
only showing top 10 rows



In [10]:
df.write.format('bigquery').option("temporaryGcsBucket","temp_bucket_temp").option('table', 'ExtendedSqlLiteDataset.ownership_transitions').partitionBy("from_address").mode('overwrite').save()

In [15]:
conn = sqlite3.connect(db_path)

In [16]:
cur = conn.execute("SELECT * from current_market_values")

rows = cur.fetchall()

In [17]:
table_columns = ['nft_address', 'token_id', 'market_value']

with open("current_market_values.csv", "w", newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(table_columns)
    writer.writerows(rows)

In [3]:
spark_current_market_values = sqlContext.read.format("csv").option("header","true").option("delimiter", ",").load("gs://nft_csvingestion/current_market_values.csv")
spark_current_market_values.show(10)

+--------------------+--------+------------------+
|         nft_address|token_id|      market_value|
+--------------------+--------+------------------+
|0x00000000000b7F8...|       0| 30000000000000000|
|0x00000000000b7F8...|       1|150000000000000000|
|0x00000000000b7F8...|      10|150000000000000000|
|0x00000000000b7F8...|      11| 30000000000000000|
|0x00000000000b7F8...|      12| 90000000000000000|
|0x00000000000b7F8...|      13| 90000000000000000|
|0x00000000000b7F8...|      14| 90000000000000000|
|0x00000000000b7F8...|      15| 30000000000000000|
|0x00000000000b7F8...|      16|150000000000000000|
|0x00000000000b7F8...|      17|150000000000000000|
+--------------------+--------+------------------+
only showing top 10 rows



In [4]:
spark_current_market_values.write.format('bigquery').option("temporaryGcsBucket","temp_bucket_temp").option('table', 'ExtendedSqlLiteDataset.current_market_values').partitionBy("nft_address").mode('overwrite').save()

In [6]:
conn = sqlite3.connect(db_path)

cur = conn.execute("SELECT * from mints")

rows = cur.fetchall()

In [7]:
table_columns = ['event_id', 'transaction_hash', 'block_number', 'nft_address', 'token_id', 'from_address', 'to_address', 'transaction_value', 'timestamp']

with open("mints.csv", "w", newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(table_columns)
    writer.writerows(rows)

In [2]:
spark_mints = sqlContext.read.format("csv").option("header","true").option("delimiter", ",").load("gs://nft_csvingestion/mints.csv")
spark_mints.show(10)

+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------+
|            event_id|    transaction_hash|block_number|         nft_address|            token_id|        from_address|          to_address|transaction_value| timestamp|
+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------+
|6c6d226e-2096-4b5...|0xe47ac62111db3a6...|    12936415|0x02AA731631c6D7F...|                3223|0x000000000000000...|0x8D745Fb3311cC86...|                0|1627776933|
|263c923f-4db9-4d8...|0xe47ac62111db3a6...|    12936415|0x02AA731631c6D7F...|                5077|0x000000000000000...|0x8D745Fb3311cC86...|                0|1627776933|
|8baa8502-42f9-453...|0xe47ac62111db3a6...|    12936415|0x02AA731631c6D7F...|                5070|0x000000000000000...|0x8D745Fb3311cC86...|          

In [3]:
spark_mints.write.format('bigquery').option("temporaryGcsBucket","temp_bucket_temp").option('table', 'ExtendedSqlLiteDataset.mints').partitionBy("nft_address").mode('overwrite').save()